In [1]:
!nvidia-smi

Fri Jun 10 01:52:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import json
import re
import string

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Install the vncorenlp python wrapper
!pip3 install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-06-10 01:52:07--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   147MB/s    in 0.2s    

2022-06-10 01:52:08 (147 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2022-06-10 01:52:08--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.co

In [4]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [5]:
# Input 
text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

# To perform word (and sentence) segmentation
sentences = rdrsegmenter.tokenize(text) 
for sentence in sentences:
	print(" ".join(sentence))

Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội .
Bà Lan , vợ ông Chúc , cũng làm_việc tại đây .


# Legal Corpus

## Read data

In [6]:
with open('/content/drive/MyDrive/NLP/Information_Retrieval/Legal_Text_Retrieval_Zalo2021/datasets/train_question_answer.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [7]:
question_answer_df = pd.DataFrame.from_dict(data['items'])
question_answer_df

,question_id,question,relevant_articles
0,0637bf82c8b290c7875c5bfddbf91df5,Công an xã xử phạt lỗi không mang bằng lái xe ...,"[{'law_id': '47/2011/tt-bca', 'article_id': '7'}]"
1,ade2b2ee4f5b869f75f0d183902382af,Phải thực hiện thao tác nạp mẫu vào bình chứa ...,"[{'law_id': '41/2020/tt-bca', 'article_id': '1..."
2,8fa164edc7a8419cd7dc9ce66d8e695a,Trình tự đánh giá chất lượng hàng năm đối với ...,"[{'law_id': '159/2020/nđ-cp', 'article_id': '1..."
3,3aed91309b72097b34427dd28bebd98a,Tử tù chết sau khi tiêm thuốc độc mà không có ...,"[{'law_id': '53/2010/qh12', 'article_id': '60'..."
4,fe9131a8862ce1bfa7de7e2b30eeb39e,Trách nhiệm của Bộ Khoa học và Công nghệ về qu...,"[{'law_id': '63/2020/nđ-cp', 'article_id': '20'}]"
...,...,...,...
3191,bd2f979dabdf0033a00ff488b2893e12,Nội dung kiểm tra công tác nghiệm thu công trì...,"[{'law_id': '06/2021/nđ-cp', 'article_id': '24'}]"
3192,b7347919f2bd65a2da6f9d0b1728a51c,Mức phạt đối với hành vi chuyển mục đích sử dụ...,"[{'law_id': '35/2019/nđ-cp', 'article_id': '12'}]"
3193,bda416f28b8c88ce25604a26b01081cb,Trích xuất theo thi hành án hình sự được hiểu ...,"[{'law_id': '41/2019/qh14', 'article_id': '3'}]"
3194,6088765b2e502c7d900a3f9a2f82f2b4,"Chạy ô tô để dầu, nhờn rơi xuống đường phạt ba...","[{'law_id': '100/2019/nđ-cp', 'article_id': '2..."


## EDA Legal Corpus

In [8]:
number = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
chars = ["a", "b", "c", "d", "đ", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o"]
stop_word = number + chars + ["của", "và", "các", "có", "được", "theo", "tại", "trong", "về", 
                              "hoặc", "người",  "này", "khoản", "cho", "không", "từ", "phải", 
                              "ngày", "việc", "sau",  "để",  "đến", "bộ",  "với", "là", "năm", 
                              "khi", "số", "trên", "khác", "đã", "thì", "thuộc", "điểm", "đồng",
                              "do", "một", "bị", "vào", "lại", "ở", "nếu", "làm", "đây", 
                              "như", "đó", "mà", "nơi", "điều"]

def preprocess(text):
    # Converting to lowercase
    text = text.lower()

    # Remove \n \t
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\t+', ' ', text)

    # Remove numbers
    text = re.sub(r'\s[0-9]+\s', ' ', text)

    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

    # Remove ""
    text = re.sub(r'(”|“)', ' ', text)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    try:
        sentences = rdrsegmenter.tokenize(text)
        text = ""
        for sentence in sentences:
            text += " ".join(sentence)
    except:
        print("Can not tokenizer: ", text)
    
    # Remove stop words
    words = text.split(' ')
    words = [word for word in words if word not in stop_word]
    text = " ".join(words)

    return text


def preprocess_df(df):
    """
    This function is for processing source of notebook
    returns preprocessed dataframe
    """
    df['question'] = df['question'].apply(preprocess)
    return df

In [9]:
clean_question_answer_df = preprocess_df(question_answer_df)

In [10]:
clean_question_answer_df['question'][:10]

0         công_an xã xử_phạt lỗi mang bằng lái_xe đúng
1    thực_hiện thao_tác nạp mẫu bình chứa xử_lý mẫu...
2    trình_tự đánh_giá chất_lượng hàng đối_với kiểm...
3    tử_tù chết tiêm thuốc_độc thân_nhân nhận xác h...
4    trách_nhiệm khoa_học công_nghệ quản_lý phát_tr...
5      đập phá biển_báo khu_vực biên_giới phạt thế_nào
6    mục_tiêu phát_triển khu công_nghiệp hỗ_trợ quy...
7    thừa phát tống_đạt những giấy_tờ hồ_sơ tài_liệ...
8    mức phạt sử_dụng trái_phép lòng_đường đô_thị h...
9    hai công_ty con chung công_ty mẹ tham_dự gói_t...
Name: question, dtype: object

In [11]:
clean_question_answer_df.to_pickle('/content/drive/MyDrive/NLP/Information_Retrieval/Legal_Text_Retrieval_Zalo2021/datasets/clean_question_answer_df.pkl')